# Getting information from SBML PBK model files

This notebook demonstrates how information can be obtained from a PBK model stored in the SBML format using the EuroMix PBK model SBML model as example.

## Preparation

In [ ]:
import pandas as pd
import roadrunner as rr
import math
import matplotlib.pyplot as plt
import libsbml as ls
from tabulate import tabulate

In [ ]:
# Specify sbml file path
sbml_file = '../model/euromix.annotated.sbml'

## Parsing the model

### General SBML file format checks

In [ ]:
# Open SBML document using libSBML
document = ls.readSBML(sbml_file)

# Check for errors
if document.getNumErrors() > 0:
    print("Encountered " + str(document.getNumErrors()) + " SBML errors:" )
    document.printErrors()

# Get SBML level and version and print
level = document.getLevel()
version = document.getVersion()
print("\n"
      + "File: " + sbml_file
      + " (Level " + str(level) + ", version " + str(version) + ")" )

# Get the model
model = document.getModel()

if model is None:
    print("No model present." )

### Describe compartments

Each compartment should be linked to a compartment matching the vocabulary of the to-be-developed PBK modelling ontology. For each compartment, the unit of measurement of the volume should be specified.

In [ ]:
# Helper function to extract is-a resource URI
def getTerm(element):
    cvTerms = element.getCVTerms()
    if not cvTerms:
        return None

    for term in cvTerms:
      num_resources = term.getNumResources()
      for j in range(num_resources):
          if term.getQualifierType() == ls.BIOLOGICAL_QUALIFIER and \
              term.getBiologicalQualifierType() == ls.BQB_IS:
              return term.getResourceURI(j)

    return None 

def getUnitString(element):
    # TODO: produce some more humanly readible unit strings
    return ls.UnitDefinition.printUnits(element.getDerivedUnitDefinition())

# Create compartments info table
print("Compartments (" + str(model.getNumCompartments()) + ")")
dt_compartments = []
for i in range(0,model.getNumCompartments()):
    c = model.getCompartment(i)
    dt_compartments.append([
        c.getId(),
        c.getName(),
        c.getVolume(),
        getUnitString(c),
        getTerm(c)
    ])
df_compartments = pd.DataFrame(
    dt_compartments,
    columns=["Id", "Name", "Volume", "Unit", "URI"]
)
print(tabulate(df_compartments, headers="keys", tablefmt="fancy_grid", showindex=False))

### Describe substances (chemical species)

Each species should represent a substance concentration/amount of a chemical substance in one of the compartments. For each species, the unit of measurement of the concentration/amount should be specified.

In [ ]:
# Create species info table
dt_species = []
print("Number of species: " + str(model.getNumSpecies()))
for i in range(0,model.getNumSpecies()):
    s = model.getSpecies(i)
    dt_species.append([
        s.getId(),
        s.getName(),
        s.getCompartment(),
        getUnitString(s)
    ])
df_species = pd.DataFrame(
    dt_species,
    columns=["Id", "Name", "IdCompartment", "Unit"]
)
print(tabulate(df_species, headers="keys", tablefmt="fancy_grid", showindex=False))

### Describe parameters

The unit for each parameter should be specied. Unitless parameters should be explicitly specified as such. Each parameter should be associated with the correct ontological term(s) that exactly identify the type of parameter so that it becomes possible to search for the parameter value in databases. How to accomplish this is yet unclear?

In [ ]:
# Create parameters info table
dt_params = []
print("Number of parameters: " + str(model.getNumParameters()))
for i in range(0,model.getNumParameters()):
    s = model.getParameter(i)
    dt_params.append([
        s.getId(),
        s.getName(),
        getUnitString(s),
        getTerm(s)
    ])
df_params = pd.DataFrame(
    dt_params,
    columns=["Id", "Name", "Unit", "URI"]
)
print(tabulate(df_params, headers="keys", tablefmt="fancy_grid", showindex=False))

### Render the differential equations

Script below renders the differential equations of the model in simple text format. Would nice to get some latex formatting for this as well.

In [ ]:
# Get the list of species IDs
species_ids = [species.getId() for species in model.getListOfSpecies()]

# Initialize dictionaries to store the differential equations and reaction rates
differential_equations = {}
reaction_rates = {}

# Iterate over the reactions in the model
for i in range(model.getNumReactions()):
    reaction = model.getReaction(i)
    
    # Get the kinetic law associated with the reaction
    kinetic_law = reaction.getKineticLaw()
    if kinetic_law is None:
        continue
    
    # Get the reaction rate formula
    rate_formula = ls.formulaToString(kinetic_law.getMath())
    
    # Iterate over the reactants and products of the reaction
    for j in range(reaction.getNumReactants()):
        reactant = reaction.getReactant(j)
        species_id = reactant.getSpecies()
        
        # Add the reaction rate to the dictionary
        if species_id in reaction_rates:
            reaction_rates[species_id].append(f"- {rate_formula}")
        else:
            reaction_rates[species_id] = [f"- {rate_formula}"]
    
    for j in range(reaction.getNumProducts()):
        product = reaction.getProduct(j)
        species_id = product.getSpecies()
        
        # Add the negative reaction rate to the dictionary
        if species_id in reaction_rates:
            reaction_rates[species_id].append(rate_formula)
        else:
            reaction_rates[species_id] = [rate_formula]

# Construct the differential equations for each species
for species_id in species_ids:
    if species_id in reaction_rates:
        new_line = '\n'
        pref = f'{new_line}{" " * (len(species_id) + 8)}'
        equation = f"d[{species_id}]/dt = {f'{pref} + '.join(reaction_rates[species_id])}"
        differential_equations[species_id] = equation.replace("+ -", "-")

# Print the differential equations for each species
for species_id, equation in differential_equations.items():
    print(equation)

### Render model diagram

Would be nice to be able to automatically generate the model diagram. The SBMLDiagrams makes an effort, but it is not really satisfactory.

In [ ]:
import SBMLDiagrams
df = SBMLDiagrams.load(sbml_file)
df.autolayout()
df.draw()

## Export annotation template

To make model annotation a bit easier, we could create an annotation template. The code below demonstrates how we could do this.

In [ ]:
df_compartments['sbml_type'] = 'compartment'
df_species['sbml_type'] = 'species'
df_params['sbml_type'] = 'parameter'
df_annotations = pd.concat(
    [
        df_compartments[['Id', 'sbml_type','Name', 'Unit']],
        df_species[['Id', 'sbml_type','Name', 'Unit']],
        df_params[['Id', 'sbml_type','Name', 'Unit']]
    ],
)
df_annotations.columns=['id', 'sbml_type','name', 'unit']
df_annotations['pattern'] = df_annotations['id']
df_annotations.insert(5, 'annotation_type', 'rdf')
df_annotations.insert(6, 'qualifier', 'BQB_IS')
df_annotations.insert(7, 'resource', '')
df_annotations.to_csv('euromix.annotations.csv', index=False)

## Running the model

In [ ]:
# Helper function for plotting simulation results
def plotSimulationResults(res, selections):
    fig, axes = plt.subplots(3,math.ceil((len(selections) - 1)/3),figsize=(10,6))
    axes = axes.flatten()
    for ax in axes:
        ax.set(xlabel='time') 
    labels = selections
    for i in range(1, len(selections)):
        sid = selections[i]
        axes[i-1].plot(res['time'], res[sid])
        axes[i-1].set(ylabel=f"Amount", title=labels[i])
    fig.tight_layout()

In [ ]:
# Specify the input
input_compartment = 'QGut'

### Single bolus

Single oral dose of 1 at $t_0$ fed as initial amount of the input compartment.

In [ ]:
# Load the PBPK model from the SBML file
rr_model = rr.RoadRunner(sbml_file)

# Set initial amout
rr_model.setInitAmount(input_compartment, 1)
rr_model.setConstant(input_compartment, False)
rr_model.setBoundary(input_compartment, False)

# Simulate the PBPK model
rr_model.resetAll()
results = rr_model.simulate(start=0, end=24, points=100)

# Plot the results
plotSimulationResults(results, rr_model.timeCourseSelections)

### Continuous dose

Continuous dose of 1 set as initial amount, accomplished by setting the input compartment to boundary and constant.

In [ ]:
# Load the PBPK model from the SBML file
rr_model = rr.RoadRunner(sbml_file)

# Set initial amout set constant and boundary for Gut
rr_model.setInitAmount(input_compartment, 1)
rr_model.setConstant(input_compartment, True)
rr_model.setBoundary(input_compartment, True)

# Simulate the PBPK model
rr_model.resetAll()
results = rr_model.simulate(start=0, end=30, points=100)

# Plot the results
plotSimulationResults(results, rr_model.timeCourseSelections)

### Run with bolus events

Bolus inputs fed to the model via events (one per day).

In [ ]:
# Load the PBPK model from the SBML file
rr_model = rr.RoadRunner(sbml_file)

# Make sure A_gut is not constant and does not have boundary conditions
rr_model.setInitAmount(input_compartment, 0)
rr_model.setConstant(input_compartment, False)
rr_model.setBoundary(input_compartment, False)

# Remove all events
for event_id in rr_model.model.getEventIds():
    rr_model.removeEvent(event_id)

# Define a list of time/dose tuples
time_dose_list = [
    (0, 1),
    (24, 2),
    (48, 1),
    (72, 2),
    (96, 3)
]

# Create an event for each of the time-dose tuples
for i in range(0, len(time_dose_list)):
    dose_event = time_dose_list[i]
    eid = f"oral_dose_event_{i}"
    rr_model.addEvent(eid, False, f"time > {dose_event[0]}", False)
    rr_model.addEventAssignment(eid, input_compartment, f"{input_compartment} + {dose_event[1]}", False)
rr_model.regenerateModel(True, True)

# Simulate the PBPK model
results = rr_model.simulate(start=0, end=5*24, points=100)

# Plot the results
plotSimulationResults(results, rr_model.timeCourseSelections)

### Run with continuous input events

Continuous input events during 8 hours of the day over multiple days. Accomplished by setting the boundary condition to true.

In [ ]:
# Load the PBPK model from the SBML file
rr_model = rr.RoadRunner(sbml_file)

# Make sure A_gut is not constant and does not have boundary conditions
rr_model.setInitAmount(input_compartment, 0)
rr_model.setConstant(input_compartment, False)
rr_model.setBoundary(input_compartment, True)

# Remove all events
for event_id in rr_model.model.getEventIds():
    rr_model.removeEvent(event_id)

# Define a list of time/dose tuples
time_dose_duration_list = [
    (0, 1, 12),
    (24, 2, 12),
    (48, 1, 12),
    (72, 2, 12),
    (96, 3, 12)
]

# Create an event for each of the time-dose tuples
for i in range(0, len(time_dose_list)):
    dose_event = time_dose_duration_list[i]
    eid = f"oral_dose_event_{i}_start"
    rr_model.addEvent(eid, False, f"time > {dose_event[0]}", False)
    rr_model.addEventAssignment(eid, input_compartment, f"{input_compartment} + {dose_event[1]}", False)
    eid = f"oral_dose_event_{i}_stop"
    rr_model.addEvent(eid, False, f"time > {dose_event[0]} + {dose_event[2]}", False)
    rr_model.addEventAssignment(eid, input_compartment, f"{input_compartment} - {dose_event[1]}", False)
rr_model.regenerateModel(True, True)

# Simulate the PBPK model
results = rr_model.simulate(start=0, end=5*24, points=100)

# Plot the results
plotSimulationResults(results, rr_model.timeCourseSelections)